In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.callbacks import History, CSVLogger
from keras.metrics import Precision, Recall
import keras.backend as K
from sklearn.metrics import roc_auc_score

import tensorflow as tf

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.RandomOverSampler.html

Using TensorFlow backend.
/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
xTrain = pd.read_pickle('../data/X_train_PREOP_10_24.pkl')
yTrain = pd.read_pickle('../data/y_train_PREOP_10_24.pkl')
xDev = pd.read_pickle('../data/X_dev_PREOP_10_24.pkl')
yDev = pd.read_pickle('../data/y_dev_PREOP_10_24.pkl')

In [3]:
xTrain.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,ArrhythDur_when_Combo_SHORT,ArrhythDur_when_Combo_LONG
0,-2.12457,0.11594,0.32942,0.15700,-2.65268,-0.35221,-2.27155,-1.62927,2.47597,0.98148,3.58585,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.08243,-0.82771,-0.95497,-0.39563,-0.10617,-0.13691,0.04597,0.25206,-0.29493,0.33103,0.49141,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.20037,1.53140,1.56497,0.35976,0.59504,-0.25533,-0.34028,0.32442,-0.67978,0.57495,-0.12748,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0.44103,-0.31888,-0.21754,-0.04732,0.59504,-0.25533,0.04597,-0.76096,-0.29493,0.16842,-0.64322,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,-0.75014,-1.05900,0.19268,0.56118,-2.17290,-0.36298,0.04597,0.39678,-0.29493,0.57495,-0.65353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0


In [4]:
xTrain.shape

(34192, 109)

In [6]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(109, input_dim=109, activation='relu')) #109 is num of features
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [47]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=10, batch_size=50, verbose=0)
kfold = StratifiedKFold(n_splits=2, shuffle=True)
results = cross_val_score(estimator, xTrain, yTrain, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 98.30% (0.00%)


In [35]:
parameters = {'batch_size': [25, 50],
          'epochs': [10, 25]}
grid_search = GridSearchCV(estimator = estimator,
                       param_grid = parameters,
                       scoring = 'accuracy',
                       cv = 10)
grid_search = grid_search.fit(xTrain, yTrain)

KeyboardInterrupt: 

## Don't use Sklearn api wrapper

In [11]:
history = History()
csv_logger = CSVLogger('./{0}_{1}.log'.format('fname', 1e-13), separator=',', append=True)
BATCH_SIZE = 50
NUM_EPOCHS = 15

In [24]:
model = create_baseline()
model.fit(xTrain, yTrain, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2)

Epoch 1/2
 - 1s - loss: 0.0946 - accuracy: 0.9821
Epoch 2/2
 - 1s - loss: 0.0832 - accuracy: 0.9830


In [7]:
from sklearn.metrics import classification_report
def getResults(model, xDev, yDev):
    yPred = model.predict(xDev, batch_size=50, verbose=1)
    y_pred_bool = yPred
    y_pred_bool[yPred > .5] = 1
    y_pred_bool[yPred <= .5] = 0
    print(roc_auc_score(yDev, yPred))
    print(classification_report(yDev, y_pred_bool))

In [8]:
model = Sequential()
model.add(Dense(60, input_dim=109, activation='relu')) #109 is num of features
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
class_weight = {0: 1.,
                1: 5000.}

In [12]:
model.fit(xTrain, yTrain, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2, class_weight=class_weight)

Epoch 1/15
 - 3s - loss: 9.3825 - accuracy: 0.0322
Epoch 2/15
 - 2s - loss: 5.2900 - accuracy: 0.0171
Epoch 3/15
 - 2s - loss: 5.1560 - accuracy: 0.0171
Epoch 4/15
 - 2s - loss: 5.0267 - accuracy: 0.0180
Epoch 5/15
 - 2s - loss: 4.9595 - accuracy: 0.0213
Epoch 6/15
 - 2s - loss: 4.9091 - accuracy: 0.0299
Epoch 7/15
 - 2s - loss: 4.6702 - accuracy: 0.0352
Epoch 8/15
 - 2s - loss: 4.5624 - accuracy: 0.0605
Epoch 9/15
 - 2s - loss: 4.4667 - accuracy: 0.0761
Epoch 10/15
 - 2s - loss: 4.2628 - accuracy: 0.1055
Epoch 11/15
 - 2s - loss: 3.9793 - accuracy: 0.1451
Epoch 12/15
 - 2s - loss: 3.8622 - accuracy: 0.1643
Epoch 13/15
 - 2s - loss: 3.5695 - accuracy: 0.2160
Epoch 14/15
 - 2s - loss: 3.3567 - accuracy: 0.2637
Epoch 15/15
 - 2s - loss: 3.3583 - accuracy: 0.2948


In [105]:
yPred = model.predict(xDev, batch_size=50, verbose=1)
yPred.mean()

4274/4274 [==============================] - 0s 11us/step


0.15957732

In [13]:
getResults(model, xDev, yDev)

4274/4274 [==============================] - 0s 24us/step
0.5574944647882272
              precision    recall  f1-score   support

           0       0.99      0.24      0.38      4201
           1       0.02      0.88      0.04        73

    accuracy                           0.25      4274
   macro avg       0.51      0.56      0.21      4274
weighted avg       0.97      0.25      0.38      4274



## Oversample

In [14]:
ros = RandomOverSampler(1, random_state = 4) #50-50 split stroke to no-stroke
X_res, y_res = ros.fit_sample(X=xTrain, y=yTrain)

In [15]:
model.fit(X_res, y_res, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2, class_weight=class_weight)

Epoch 1/15
 - 3s - loss: 2.5771 - accuracy: 0.6235
Epoch 2/15
 - 3s - loss: 1.7166 - accuracy: 0.7412
Epoch 3/15
 - 3s - loss: 1.0881 - accuracy: 0.8402
Epoch 4/15
 - 3s - loss: 0.7093 - accuracy: 0.8979
Epoch 5/15
 - 3s - loss: 0.5453 - accuracy: 0.9220
Epoch 6/15
 - 3s - loss: 0.3680 - accuracy: 0.9481
Epoch 7/15
 - 3s - loss: 0.3088 - accuracy: 0.9559
Epoch 8/15
 - 4s - loss: 9.4163 - accuracy: 0.6565
Epoch 9/15
 - 3s - loss: 3.5730 - accuracy: 0.7053
Epoch 10/15
 - 3s - loss: 1.9807 - accuracy: 0.7810
Epoch 11/15
 - 3s - loss: 1.2229 - accuracy: 0.8442
Epoch 12/15
 - 3s - loss: 1.0106 - accuracy: 0.8870
Epoch 13/15
 - 3s - loss: 0.7266 - accuracy: 0.9096
Epoch 14/15
 - 3s - loss: 0.4752 - accuracy: 0.9391
Epoch 15/15
 - 3s - loss: 0.3305 - accuracy: 0.9562


In [16]:
getResults(model, xDev, yDev)

4274/4274 [==============================] - 0s 63us/step
0.5158214123838747
              precision    recall  f1-score   support

           0       0.98      0.91      0.94      4201
           1       0.02      0.12      0.04        73

    accuracy                           0.89      4274
   macro avg       0.50      0.52      0.49      4274
weighted avg       0.97      0.89      0.93      4274



### Maybe try with tree data?

In [17]:
xTrainTree = pd.read_pickle('../data/X_train_PREOP_TREE_10_24.pkl')
yTrainTree = pd.read_pickle('../data/y_train_PREOP_TREE_10_24.pkl')
xDevTree = pd.read_pickle('../data/X_dev_PREOP_TREE_10_24.pkl')
yDevTree = pd.read_pickle('../data/y_dev_PREOP_TREE_10_24.pkl')

In [18]:
model = Sequential()
model.add(Dense(60, input_dim=73, activation='relu')) #109 is num of features
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
class_weight = {0: 1.,
                1: 60.}

In [20]:
model.fit(xTrainTree, yTrainTree, epochs=5, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2, class_weight=class_weight)

Epoch 1/5
 - 3s - loss: 1.3701 - accuracy: 0.5611
Epoch 2/5
 - 3s - loss: 1.2845 - accuracy: 0.6077
Epoch 3/5
 - 3s - loss: 1.2617 - accuracy: 0.6215
Epoch 4/5
 - 3s - loss: 1.2289 - accuracy: 0.6136
Epoch 5/5
 - 3s - loss: 1.1893 - accuracy: 0.6294


In [21]:
getResults(model, xDevTree, yDevTree)

4274/4274 [==============================] - 0s 39us/step
0.6483257410988251
              precision    recall  f1-score   support

           0       0.99      0.52      0.68      4201
           1       0.03      0.78      0.05        73

    accuracy                           0.52      4274
   macro avg       0.51      0.65      0.37      4274
weighted avg       0.98      0.52      0.67      4274



In [155]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(yTrain),
                                                 yTrain)

In [156]:
class_weights

array([ 0.50865814, 29.37457045])

In [157]:
class_weights_tree = class_weight.compute_class_weight('balanced',
                                                 np.unique(yTrainTree),
                                                 yTrainTree)

In [158]:
class_weights_tree

array([ 0.50865814, 29.37457045])

In [22]:
#https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [23]:
def buildModel(loss, dropoutRate):
    model = Sequential()
    model.add(Dense(120, input_dim=73, activation='relu')) #73 is num of features
    model.add(Dropout(dropoutRate))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(dropoutRate))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy', f1])
    return model

In [24]:
model = buildModel(f1_loss, 0.1)
class_weight = {0: 1.,
                1: 60.}
model.fit(xTrainTree, yTrainTree, epochs=15, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2, class_weight=class_weight)

Epoch 1/15
 - 5s - loss: 1.8816 - accuracy: 0.8019 - f1: 0.0421
Epoch 2/15
 - 3s - loss: 1.8552 - accuracy: 0.7449 - f1: 0.0505
Epoch 3/15
 - 3s - loss: 1.8362 - accuracy: 0.7554 - f1: 0.0532
Epoch 4/15
 - 4s - loss: 1.8355 - accuracy: 0.7583 - f1: 0.0541
Epoch 5/15
 - 3s - loss: 1.8260 - accuracy: 0.8303 - f1: 0.0574
Epoch 6/15
 - 3s - loss: 1.8365 - accuracy: 0.8738 - f1: 0.0553
Epoch 7/15
 - 3s - loss: 1.8119 - accuracy: 0.8478 - f1: 0.0623
Epoch 8/15
 - 4s - loss: 1.8309 - accuracy: 0.7869 - f1: 0.0598
Epoch 9/15
 - 3s - loss: 1.8434 - accuracy: 0.6921 - f1: 0.0526
Epoch 10/15
 - 3s - loss: 1.8305 - accuracy: 0.7442 - f1: 0.0596
Epoch 11/15
 - 3s - loss: 1.8474 - accuracy: 0.5894 - f1: 0.0510
Epoch 12/15
 - 3s - loss: 1.8791 - accuracy: 0.7999 - f1: 0.0399
Epoch 13/15
 - 3s - loss: 1.8598 - accuracy: 0.7763 - f1: 0.0447
Epoch 14/15
 - 3s - loss: 1.8248 - accuracy: 0.7847 - f1: 0.0623
Epoch 15/15
 - 3s - loss: 1.8449 - accuracy: 0.7389 - f1: 0.0522


In [25]:
getResults(model, xDevTree, yDevTree)

4274/4274 [==============================] - 0s 54us/step
0.5435447528801036
              precision    recall  f1-score   support

           0       0.98      0.84      0.91      4201
           1       0.03      0.25      0.05        73

    accuracy                           0.83      4274
   macro avg       0.51      0.54      0.48      4274
weighted avg       0.97      0.83      0.89      4274



In [26]:
model = buildModel('binary_crossentropy', 0.1)
class_weight = {0: 1.,
                1: 60.}
model.fit(xTrainTree, yTrainTree, epochs=15, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2, class_weight=class_weight)

Epoch 1/15
 - 4s - loss: 1.3765 - accuracy: 0.5318 - f1: 0.0394
Epoch 2/15
 - 3s - loss: 1.3181 - accuracy: 0.5709 - f1: 0.0497
Epoch 3/15
 - 3s - loss: 1.2899 - accuracy: 0.5525 - f1: 0.0512
Epoch 4/15
 - 3s - loss: 1.2673 - accuracy: 0.5657 - f1: 0.0532
Epoch 5/15
 - 3s - loss: 1.2378 - accuracy: 0.5470 - f1: 0.0532
Epoch 6/15
 - 4s - loss: 1.2166 - accuracy: 0.5853 - f1: 0.0549
Epoch 7/15
 - 4s - loss: 1.1879 - accuracy: 0.5953 - f1: 0.0583
Epoch 8/15
 - 5s - loss: 1.1674 - accuracy: 0.5914 - f1: 0.0598
Epoch 9/15
 - 4s - loss: 1.1266 - accuracy: 0.6285 - f1: 0.0654
Epoch 10/15
 - 3s - loss: 1.0896 - accuracy: 0.6423 - f1: 0.0687
Epoch 11/15
 - 3s - loss: 1.0538 - accuracy: 0.6425 - f1: 0.0715
Epoch 12/15
 - 3s - loss: 1.0291 - accuracy: 0.6547 - f1: 0.0732
Epoch 13/15
 - 3s - loss: 0.9786 - accuracy: 0.6851 - f1: 0.0782
Epoch 14/15
 - 3s - loss: 0.9412 - accuracy: 0.7027 - f1: 0.0845
Epoch 15/15
 - 3s - loss: 0.8926 - accuracy: 0.7359 - f1: 0.0918


In [27]:
getResults(model, xDevTree, yDevTree)

4274/4274 [==============================] - 0s 84us/step
0.5604063611729758
              precision    recall  f1-score   support

           0       0.99      0.68      0.81      4201
           1       0.02      0.44      0.04        73

    accuracy                           0.68      4274
   macro avg       0.50      0.56      0.43      4274
weighted avg       0.97      0.68      0.79      4274



In [28]:
model = buildModel('binary_crossentropy', 0)
class_weight = {0: 1.,
                1: 60.}
model.fit(xTrainTree, yTrainTree, epochs=15, batch_size=BATCH_SIZE, shuffle=True, 
          callbacks=[history, csv_logger], verbose=2, class_weight=class_weight)

Epoch 1/15
 - 3s - loss: 1.3717 - accuracy: 0.5839 - f1: 0.0381
Epoch 2/15
 - 3s - loss: 1.2975 - accuracy: 0.5745 - f1: 0.0505
Epoch 3/15
 - 3s - loss: 1.2748 - accuracy: 0.5766 - f1: 0.0507
Epoch 4/15
 - 3s - loss: 1.2404 - accuracy: 0.5883 - f1: 0.0543
Epoch 5/15
 - 3s - loss: 1.2232 - accuracy: 0.5917 - f1: 0.0569
Epoch 6/15
 - 3s - loss: 1.1789 - accuracy: 0.6152 - f1: 0.0597
Epoch 7/15
 - 4s - loss: 1.1406 - accuracy: 0.6411 - f1: 0.0662
Epoch 8/15
 - 5s - loss: 1.0960 - accuracy: 0.6429 - f1: 0.0680
Epoch 9/15
 - 4s - loss: 1.0454 - accuracy: 0.6861 - f1: 0.0744
Epoch 10/15
 - 4s - loss: 0.9797 - accuracy: 0.7067 - f1: 0.0825
Epoch 11/15
 - 3s - loss: 0.9220 - accuracy: 0.7243 - f1: 0.0912
Epoch 12/15
 - 4s - loss: 0.8755 - accuracy: 0.7414 - f1: 0.0952
Epoch 13/15
 - 4s - loss: 0.8293 - accuracy: 0.7470 - f1: 0.1031
Epoch 14/15
 - 3s - loss: 0.7723 - accuracy: 0.7651 - f1: 0.1068
Epoch 15/15
 - 3s - loss: 0.7259 - accuracy: 0.7705 - f1: 0.1116


In [29]:
getResults(model, xDevTree, yDevTree)

4274/4274 [==============================] - 0s 31us/step
0.5355166578081539
              precision    recall  f1-score   support

           0       0.98      0.81      0.89      4201
           1       0.02      0.26      0.04        73

    accuracy                           0.80      4274
   macro avg       0.50      0.54      0.47      4274
weighted avg       0.97      0.80      0.87      4274

